<a href="https://colab.research.google.com/github/Anastasiamay/Tinkoff/blob/main/sales_funnel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Построение воронки этапов процесса

### Описание
Необходимо построить воронку для проекта "Тинькофф квест", которая будет отображать динамику по неделям и месяцам, сколько клиентов посещает сайт, сколько из них регистрируются на нем, подают заявку на игру и фактически приходят на нее

### Цель
Изучить конверсию на разных этапах взаимодействия клиента с продуктом, выявить проблемные места в процессе и определить точки роста продукта.

Импорт библиотек, подключение к базе данных

In [ ]:
import psycopg2
import pandas as pd
import plotly.graph_objs as go

In [ ]:
conn = psycopg2.connect(
    host="158.160.52.106",
    port=5432,
    database="postgres",
    user="student",
    password="JvLda93aA",
)
cur = conn.cursor()
cur.execute("SELECT * FROM msu_analytics.game")
data = cur.fetchall()

Запрос для формирования таблицы с данными

In [ ]:
df = pd.read_sql("SELECT distinct c.client_rk, \
            date_trunc('month', visit_dttm) as visit_month, \
            date_trunc('week', visit_dttm) as visit_week, \
            registration_dttm as registration, \
            application_dttm as application, \
            game_dttm as game, \
            game_flg\
            FROM msu_analytics.client c \
            left join msu_analytics.account a on a.client_rk = c.client_rk\
            left join msu_analytics.application ap on ap.account_rk = a.account_rk\
            left join msu_analytics.game g on g.game_rk = ap.game_rk\
            order by c.client_rk", conn)
df.head()

/usr/local/lib/python3.9/dist-packages/pandas/io/sql.py:762: UserWarning:

pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy



,client_rk,visit_month,visit_week,registration,application,game,game_flg
0,1,2022-09-01,2022-09-26,NaT,NaT,NaT,NaN
1,2,2022-10-01,2022-10-17,NaT,NaT,NaT,NaN
2,3,2022-09-01,2022-09-19,2022-09-26 04:34:12.392193,NaT,NaT,NaN
3,4,2022-11-01,2022-11-28,2022-12-01 00:58:56.483252,2022-12-10 16:45:58.114216,2022-12-14 12:04:00.773186,1.0
4,4,2022-11-01,2022-11-28,2022-12-01 00:58:56.483252,2022-12-13 14:14:16.594724,2023-01-02 05:14:31.462798,1.0
5,5,2023-01-01,2022-12-26,NaT,NaT,NaT,NaN
6,6,2022-11-01,2022-11-28,2022-11-29 10:41:52.466866,NaT,NaT,NaN
7,7,2022-12-01,2022-12-19,NaT,NaT,NaT,NaN
8,8,2022-12-01,2022-12-05,2022-12-07 13:34:02.310052,NaT,NaT,NaN
9,9,2022-10-01,2022-10-24,2022-10-26 20:06:10.054936,NaT,NaT,NaN


Таблицы с количеством клиентов на каждом этапе по месяцам и неделям

In [ ]:
month_funnel = df.groupby("visit_month").agg(
    {"client_rk": "nunique",
     "registration": lambda x: x.notna().sum(),
     "application": lambda x: x.notna().sum(),
     "game": lambda x: (x.notna() & (df["game_flg"] == 1)).sum()}
).reset_index()

week_funnel = df.groupby("visit_week").agg(
    {"client_rk": "nunique",
     "registration": lambda x: x.notna().sum(),
     "application": lambda x: x.notna().sum(),
     "game": lambda x: (x.notna() & (df["game_flg"] == 1)).sum()}
).reset_index()

Воронка конверсии по месяцам

In [ ]:
fig = go.Figure()

for month in month_funnel['visit_month']:
    funnel_data = month_funnel[month_funnel['visit_month'] == month]
    
    fig.add_trace(go.Funnel(
        name = month.strftime('%Y-%m'),
        y = ["Посещение сайта", "Регистрация на сайте", "Подача заявки на игру", "Посещение игры"],
        x = funnel_data.values[0][1:],
        textinfo = "value+percent initial"
    ))
fig.update_layout(title="Воронка конверсии этапов по месяцам")
fig.show()

Отметим, что данные охватывают не полный февраль, не будем учитывать этот месяц в анализе.

Конверсия всех месяцев из посещения сайта в другие этапы примерно одинаковая и составляет:
* ~30% в регистрацию на сайте;
* ~13% в заявку на игру;
* ~8% в посещение игры для месяцев сентябрь, октябрь и ноябрь. Конверсия этого этапа в декабре(4,8%) и январе(2.5%) ниже

Если рассматривать конверсию из предыдущего этапа, для всех месяцев она также примерно одинакова и составляет:
* ~30% из посещения сайта в регистрацию на сайте;
* ~40% из регистрации на сайте в заявку на игру;
* ~68% из подачи заявки в посещение игры для месяцев сентябрь, октябрь и ноябрь. Конверсия этого этапа в декабре(39%) и январе(18.9%) ниже

График конверсии из посещения сайта в этапы по неделям

In [ ]:
lines = [
    go.Scatter(
        x=week_funnel['visit_week'],
        y=round(week_funnel['registration'] / week_funnel['client_rk'] * 100, 2),
        name='Конверсия из посещения сайта в регистрацию'
    ),
    go.Scatter(
        x=week_funnel['visit_week'],
        y=round(week_funnel['application'] / week_funnel['client_rk'] * 100, 2),
        name='Конверсия из посещения сайта в заявку на игру'
    ),
    go.Scatter(
        x=week_funnel['visit_week'],
        y=round(week_funnel['game'] / week_funnel['client_rk'] * 100, 2),
        name='Конверсия из посещения сайта в посещение игры'
    )
]

layout = go.Layout(
    title='Конверсия из посещения сайта в этапы по неделям',
    xaxis=dict(title='Недели'),
    yaxis=dict(title='Конверсия')
)

fig = go.Figure(data=lines, layout=layout)

fig.show()

Видим достаточно сильный разрыв между конверсией из посещения сайта в регистрацию и другими конверсиями. На этом этапе теряется много потенциальных клиентов

График конверсии переходов в этапы по неделям

In [ ]:
lines = [
    go.Scatter(
        x=week_funnel['visit_week'],
        y=round(week_funnel['registration'] / week_funnel['client_rk'] * 100, 2),
        name='Конверсия из посещения в регистрацию'
    ),
    go.Scatter(
        x=week_funnel['visit_week'],
        y=round(week_funnel['application'] / week_funnel['registration'] * 100, 2),
        name='Конверсия из регистрации в заявку на игру'
    ),
    go.Scatter(
        x=week_funnel['visit_week'],
        y=round(week_funnel['game'] / week_funnel['application'] * 100, 2),
        name='Конверсия из заявки на игру в посещение игры'
    )
]

layout = go.Layout(
    title='Конверсия переходов в этапы по неделям',
    xaxis=dict(title='Недели'),
    yaxis=dict(title='Конверсия')
)

fig = go.Figure(data=lines, layout=layout)

fig.show()

Конверсия из посещения сайта в регистрацию и конверсия из регистрации в заявку на игру достаточно стабильны (неделя с 30 января уже относится к февралю, а данные за эту неделю февраля уже неполные, поэтому не учитываем снижение конверсии). А вот конверсия из заявки в посещение игры со второй половины декабря заметно снижается, это может обуславливаться праздниками, клиенты не расчитали свою занятость в связи с праздниками и не смогли попасть на игру

# Выводы и рекомендации

Есть выраженный сезонный характер посещения игры в новогодние праздники. Для повышения конверсии этого этапа можно рассмотреть следующие варианты:
* если при заявке на игру, клиент не оплачивает игру, то сделать либо полную оплату предстоящей игры и частичный возврат при ее отмене/не посещении, либо частичную предоплату без ее возвращения при отмене/не посещении игры;
* ввести систему накопительных бонусов или скидок на последующие игры, которые будут выше в новогодние праздники;
* ввести новогоднюю акцию в виде скидки на игру;
* добавить в игру новогоднюю тематику;
* создать новую игру, доступную только в новогодние праздники;
* рассылать уведомления/письма на электронную почту о предстоящей игре и бонусах.

Заметна потеря большого количества потенциальных клиентов на этапе регистрации после посещения сайта. Для повышения конверсии этого этапа можно рассмотреть следующие варианты:
* проверить/скорректировать удобство блока регистрации на сайте;
* обеспечить более простой, интуитивно понятный и привлекательный интерфейс, улучшить навигацию по сайту;
* упростить процесс регистрации, уменьшить количество обязательных полей;
* оптимизировать сайт для разных устройств: ПК, смартфоны, планшеты;
* добавить всплывающее уведомление при попытке покинуть сайт с формой регистрации;
* добавить отзывы клиентов на главную страницу сайта;
* возможно, клиентам не хватает уникальных игр, которых нет у конкурентов;
* предоставлять скидку на первую игру;
* предоставлять скидку на игру за "приведение друга".